# Autoscaling

## Deploy a pre-trained model to an endpoint

In [ ]:
import boto3
import time, random, uuid
import pandas as pd
import numpy as np
from sagemaker import Model
from sagemaker import image_uris
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

s3_bucket = "INSERT_S3_BUCKET"
model_artifact_s3_key = "INSERT_S3_KEY"
sagemaker_role = "INSERT_ARN"

s3_bucket = "adgu-datasets"
model_artifact_s3_key = "output/xgboost-training-1741995855/output/model.tar.gz"
sagemaker_role = "arn:aws:iam::146868985163:role/SageMaker-ExecutionRole"


model_artifact_s3 = f"s3://{s3_bucket}/{model_artifact_s3_key}"

# Initialize Boto3 SageMaker client
sagemaker_client = boto3.client("sagemaker", region_name="us-east-1")
xgboost_image_uri = image_uris.retrieve(framework='xgboost',region='us-east-1', version='1.7-1')


# **Create a SageMaker Model**
model_name = f"xgboost-model-{int(time.time())}"
print("Creating model in SageMaker named: {}".format(model_name))
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        "Image": xgboost_image_uri,
        "ModelDataUrl": model_artifact_s3,
    },
    ExecutionRoleArn=sagemaker_role,
)


# **Deploy the Model as an Endpoint**
# Create the endpoint configuration
endpoint_config_name = f"xgboost-endpoint-config-{int(time.time())}"
endpoint_name = f"static-endpoint-name"

# Create Endpoint Configuration
print("Creating endpoint configuration...")
sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "XGBoostVariant1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
        }
    ],
)

# **Deploy the Model as an Endpoint**
# Create Endpoint
print("Deploying model as an endpoint...")
sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

# Wait for endpoint to be ready
print("Waiting for endpoint to be ready...")
while True:
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = response["EndpointStatus"]
    if status in ["InService", "Failed"]:
        print(f"Endpoint Status: {status}")
        break
    time.sleep(10)

# Check if deployment was successful
if status != "InService":
    raise Exception(f"Deployment failed with status: {status}")

print(f"Model deployed successfully at endpoint: {endpoint_name}")


### Head into the web console and enable auto-scaling on the endpoint variant.

## Generate Traffic

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
# Create a Predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer(),  # Ensures input is formatted as CSV
    deserializer=JSONDeserializer(),  # Parses JSON output
)

for _ in range(1000):
    # Sample input data (excluding the target column)
    sample_data = [[30, 1, 12345]]  # Must be a 2D list
    
    # Invoke the endpoint
    prediction = predictor.predict(sample_data)
    
    print("Prediction response:", prediction)